# Bien penser à changer le CSV et le nom des DF !!

In [1]:
from selenium import webdriver
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import re, time, random
import plotly.express as px

/Users/amauryguilbaud/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
df_data_villes_clean9 = pd.read_csv("/Users/amauryguilbaud/Documents/Code/vscode/Meilleur agent/output_9.csv", 
                                    delimiter=",",
                                    index_col=0)


In [4]:
df_data_villes_clean9.shape

(2044, 2)

In [ ]:
def extract_number(text):
    """Nettoyer le texte et transformer en float"""
    clean_text = text.replace("\u202f", "").replace("\xa0", "").replace("€", "").strip()
    clean_text = clean_text.replace(",", ".")
    return float(clean_text)

# Selenium setup (Chrome auto-managed by Selenium Manager; Chrome must be installed)
options = webdriver.ChromeOptions()
options.add_argument("--lang=fr-FR")
options.add_argument("--disable-blink-features=AutomationControlled")
# Uncomment to persist cookies/session between runs:
# options.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Local\Google\Chrome\User Data\ScrapeProfile")

driver = webdriver.Chrome(options=options)
stealth(driver,
    languages=["fr-FR", "fr"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)
driver.set_window_size(1366, 768)

# Liste pour stocker toutes les données
all_data = []
error_data = []

for index, row in df_data_villes_clean9.iterrows():
    url = row['url']
    try:
        # Human-like pacing
        time.sleep(random.uniform(1.8, 3.6))

        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        data_dict = {}
        # 1-Ajouter la ville
        title_text = soup.find("title").get_text().strip()
        ville = title_text.split("Prix immobilier")[1].split("(")[0].strip()
        data_dict["ville"] = ville
        
        #2-ajouter prix achat+loyer au m2 appt et maison
        raw_text3 = soup.find_all("ul", class_="prices-summary__price-range")
        for ul in raw_text3:
            li_items = ul.find_all("li")
            label = li_items[0].get_text().strip().lower()
            main_value = extract_number(li_items[1].get_text())
            # Nettoyer le texte pour récupérer min et max correctement
            range_text = li_items[2].get_text()
            range_text = range_text.replace("\u202f", "").replace("\xa0", "").replace(",", ".")
            # Extraire tous les nombres sous forme de float
            numbers_in_range = [float(n) for n in re.findall(r"[\d\.]+", range_text)]
            if "prix" in label:
                if "prix_appartement" not in data_dict:
                    data_dict["prix_appartement"] = main_value
                    data_dict["min_appartement"] = numbers_in_range[0]
                    data_dict["max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["prix_maison"] = main_value
                    data_dict["min_maison"] = numbers_in_range[0]
                    data_dict["max_maison"] = numbers_in_range[1]
            elif "loyer" in label:
                if "loyer_appartement" not in data_dict:
                    data_dict["loyer_appartement"] = main_value
                    data_dict["loyer_min_appartement"] = numbers_in_range[0]
                    data_dict["loyer_max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["loyer_maison"] = main_value
                    data_dict["loyer_min_maison"] = numbers_in_range[0]
                    data_dict["loyer_max_maison"] = numbers_in_range[1]
        
        #3-Ajouter l'URL pour référence
        data_dict["url"] = url
        all_data.append(data_dict)
        print(f"Datas ajoutées pour la ville : {ville}")
        
    except Exception as erreur:
        print(f"Erreur pour l'URL {url}: {erreur}")
        error_data.append({
        "url": url,
        "erreur": str(erreur)})
        
    # Pause pour ne pas surcharger le site
    time.sleep(random.uniform(2.3, 3.6))

driver.quit()



Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/lacepede-47360/: list index out of range
Datas ajoutées pour la ville : Lachapelle
Datas ajoutées pour la ville : Lafitte-sur-Lot
Datas ajoutées pour la ville : Lafox
Datas ajoutées pour la ville : Lagarrigue


KeyboardInterrupt: 

In [16]:
df_error_url = pd.DataFrame(error_data)
df_meilleur_agent9 = pd.DataFrame(all_data)

In [17]:
df_error_url

,url,erreur
0,https://www.meilleursagents.com/prix-immobilie...,list index out of range


In [47]:
df_meilleur_agent9['ratio_appt'] = round((df_meilleur_agent9['prix_appartement'] / df_meilleur_agent9['loyer_appartement']),1)

df_meilleur_agent9['ratio_maison'] = round((df_meilleur_agent9['prix_maison'] / df_meilleur_agent9['loyer_maison']),1)

df_meilleur_agent9 = df_meilleur_agent9.sort_values('ratio_appt')